<a href="https://colab.research.google.com/github/KatBotkowska/WiT_workshops/blob/main/chocolate_bar_ratings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import io
from unicodedata import normalize

In [2]:
#podłączenie google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


#Pobranie danych z Kaggle - nie odpalać

In [3]:
root_path = 'gdrive/My Drive/WiT_workshops/'

In [4]:
cd 'gdrive/My Drive/WiT_workshops/'

/content/gdrive/My Drive/WiT_workshops


In [5]:
from google.colab import files
files.upload()  #this will prompt you to upload the kaggle.json

Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"kbotkowska","key":"90ebb11108c952959800f5549af3472d"}'}

In [6]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json  # set permission

kaggle.json


In [7]:
!kaggle datasets download -d rtatman/chocolate-bar-ratings
# !kaggle datasets download -d sid321axn/beijing-multisite-airquality-data-set

chocolate-bar-ratings.zip: Skipping, found more recently modified local copy (use --force to force download)


In [8]:
!unzip -q chocolate-bar-ratings.zip -d [datasets]

replace [datasets]/flavors_of_cacao.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y


#Czyszczenie danych

In [69]:
choco = pd.read_csv('/content/gdrive/MyDrive/WiT_workshops/[datasets]/flavors_of_cacao.csv', encoding='utf-8')

In [70]:
choco.head()

,Company \n(Maker-if known),Specific Bean Origin\nor Bar Name,REF,Review\nDate,Cocoa\nPercent,Company\nLocation,Rating,Bean\nType,Broad Bean\nOrigin
0,A. Morin,Agua Grande,1876,2016,63%,France,3.75,,Sao Tome
1,A. Morin,Kpime,1676,2015,70%,France,2.75,,Togo
2,A. Morin,Atsane,1676,2015,70%,France,3.00,,Togo
3,A. Morin,Akata,1680,2015,70%,France,3.50,,Togo
4,A. Morin,Quilla,1704,2015,70%,France,3.50,,Peru


In [71]:
choco.columns

Index(['Company \n(Maker-if known)', 'Specific Bean Origin\nor Bar Name',
       'REF', 'Review\nDate', 'Cocoa\nPercent', 'Company\nLocation', 'Rating',
       'Bean\nType', 'Broad Bean\nOrigin'],
      dtype='object')

In [72]:
#usunięcie znaku nowej linii \n z nazw kolumn
choco.columns = [column.replace('\n', '') for column in choco.columns.values]
choco.columns

Index(['Company (Maker-if known)', 'Specific Bean Originor Bar Name', 'REF',
       'ReviewDate', 'CocoaPercent', 'CompanyLocation', 'Rating', 'BeanType',
       'Broad BeanOrigin'],
      dtype='object')

In [73]:
choco.columns = ['Company', 'SpecificBeanOrigin', 'ReviewDataReference',
       'ReviewDate', 'CocoaPercent', 'CompanyLocation', 'Rating', 'BeanType',
       'BroadBeanOrigin']

In [74]:
choco.columns

Index(['Company', 'SpecificBeanOrigin', 'ReviewDataReference', 'ReviewDate',
       'CocoaPercent', 'CompanyLocation', 'Rating', 'BeanType',
       'BroadBeanOrigin'],
      dtype='object')

In [75]:
choco.shape

(1795, 9)

In [76]:
choco.dtypes

Company                 object
SpecificBeanOrigin      object
ReviewDataReference      int64
ReviewDate               int64
CocoaPercent            object
CompanyLocation         object
Rating                 float64
BeanType                object
BroadBeanOrigin         object
dtype: object

In [77]:
choco.describe()

,ReviewDataReference,ReviewDate,Rating
count,1795.000000,1795.000000,1795.000000
mean,1035.904735,2012.325348,3.185933
std,552.886365,2.927210,0.478062
min,5.000000,2006.000000,1.000000
25%,576.000000,2010.000000,2.875000
50%,1069.000000,2013.000000,3.250000
75%,1502.000000,2015.000000,3.500000
max,1952.000000,2017.000000,5.000000


In [78]:
choco.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1795 entries, 0 to 1794
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Company              1795 non-null   object 
 1   SpecificBeanOrigin   1795 non-null   object 
 2   ReviewDataReference  1795 non-null   int64  
 3   ReviewDate           1795 non-null   int64  
 4   CocoaPercent         1795 non-null   object 
 5   CompanyLocation      1795 non-null   object 
 6   Rating               1795 non-null   float64
 7   BeanType             1794 non-null   object 
 8   BroadBeanOrigin      1794 non-null   object 
dtypes: float64(1), int64(2), object(6)
memory usage: 126.3+ KB


In [79]:
 #ile pustych danych (nulli) mamy w tabeli?
choco.isnull().sum()

Company                0
SpecificBeanOrigin     0
ReviewDataReference    0
ReviewDate             0
CocoaPercent           0
CompanyLocation        0
Rating                 0
BeanType               1
BroadBeanOrigin        1
dtype: int64

In [80]:
#usuwa wiersze z pustymi danymi - tutaj są tylko 2 takie wiersze
choco.dropna(inplace=True)

In [81]:
 #ile pustych danych (nulli) mamy w tabeli?
choco.isnull().sum()

Company                0
SpecificBeanOrigin     0
ReviewDataReference    0
ReviewDate             0
CocoaPercent           0
CompanyLocation        0
Rating                 0
BeanType               0
BroadBeanOrigin        0
dtype: int64

In [82]:
#sortowanie danych
choco.sort_values(['Rating', 'CocoaPercent'], ascending=False).head()

,Company,SpecificBeanOrigin,ReviewDataReference,ReviewDate,CocoaPercent,CompanyLocation,Rating,BeanType,BroadBeanOrigin
78,Amedei,Chuao,111,2007,70%,Italy,5.0,Trinitario,Venezuela
86,Amedei,Toscano Black,40,2006,70%,Italy,5.0,Blend,
1528,Soma,Crazy 88,1077,2013,88%,Canada,4.0,,"Guat., D.R., Peru, Mad., PNG"
1335,Pralus,Fortissima,93,2006,80%,France,4.0,"Criollo, Trinitario",Ecuador
1427,Scharffen Berger,San Juan de Cheni,959,2012,78%,U.S.A.,4.0,,Peru


In [83]:
choco.loc[1427]['BeanType']

'\xa0'

In [84]:
funkcja1 = lambda x: x.replace(u'\xa0', u'')
choco['BeanType'] = choco['BeanType'].apply(funkcja1)

In [85]:
choco.loc[1427]['BeanType']

''

In [86]:
choco.isnull().sum()

Company                0
SpecificBeanOrigin     0
ReviewDataReference    0
ReviewDate             0
CocoaPercent           0
CompanyLocation        0
Rating                 0
BeanType               0
BroadBeanOrigin        0
dtype: int64

In [87]:
choco.isna().sum()

Company                0
SpecificBeanOrigin     0
ReviewDataReference    0
ReviewDate             0
CocoaPercent           0
CompanyLocation        0
Rating                 0
BeanType               0
BroadBeanOrigin        0
dtype: int64

In [88]:
choco['BeanType']

0                
1                
2                
3                
4                
          ...    
1790             
1791    Forastero
1792    Forastero
1793             
1794             
Name: BeanType, Length: 1793, dtype: object

In [89]:
choco.columns

Index(['Company', 'SpecificBeanOrigin', 'ReviewDataReference', 'ReviewDate',
       'CocoaPercent', 'CompanyLocation', 'Rating', 'BeanType',
       'BroadBeanOrigin'],
      dtype='object')

In [91]:
chocoMelt = choco.melt(id_vars=['CompanyLocation','Company', 'SpecificBeanOrigin'])  #zmiana kolumn na wiersze - tworzy nową zmienną variable zawierającą nazwy kolumn, id_vars - wartości które są bez zmian, value - wartości dla zmiennej variable
chocoMelt.sort_values(['CompanyLocation', 'Company'], ascending=False).sample(10)

,CompanyLocation,Company,SpecificBeanOrigin,variable,value
10022,Vietnam,Marou,Tien Giang,BroadBeanOrigin,Vietnam
2779,Colombia,Luker,Misterio,ReviewDate,2010
8549,U.S.A.,Ritual,"Camino Verde P., 2012, Balao, Guayas",BeanType,Nacional (Arriba)
8729,Canada,Soul,Ecuador,BeanType,
918,U.S.A.,L.A. Burdick (Felchlin),Brazil,ReviewDataReference,891
2744,France,Laia aka Chat-Noir,Trinidad-Tobago,ReviewDate,2014
9454,U.S.A.,Dandelion,Patanemo,BroadBeanOrigin,Venezuela
4107,South Africa,De Villiers,Bundibugyo District,CocoaPercent,70%
5909,France,Debauve & Gallais (Michel Cluizel),Venezuela,Rating,3.75
2490,U.S.A.,Fresco,"Conacado, #213, DR, -C",ReviewDate,2011


In [98]:
chocoPivot = choco.pivot_table(values='Rating', index='CompanyLocation', columns='CocoaPercent', aggfunc=pd.Series.mean)
chocoPivot

CocoaPercent,100%,42%,46%,50%,53%,55%,56%,57%,58%,60%,60.5%,61%,62%,63%,64%,65%,66%,67%,68%,69%,70%,71%,72%,72.5%,73%,73.5%,74%,75%,76%,77%,78%,79%,80%,81%,82%,83%,84%,85%,86%,87%,88%,89%,90%,91%,99%
CompanyLocation,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Amsterdam,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.500000,NaN,3.500000,NaN,NaN,NaN,NaN,NaN,NaN,3.250000,NaN,NaN,NaN,NaN,3.750000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Argentina,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.500000,NaN,3.250000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.750000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.75,NaN,NaN,NaN,NaN
Australia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.250000,3.312500,3.500000,3.437500,3.416667,NaN,3.336957,NaN,3.321429,NaN,NaN,NaN,4.000000,3.375000,NaN,NaN,3.750000,NaN,3.250000,NaN,NaN,2.750,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Austria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.500,NaN,NaN,NaN,3.50,NaN,NaN,3.250000,NaN,NaN,3.250000,NaN,3.392857,NaN,3.500000,NaN,NaN,NaN,3.250000,3.000000,2.750000,NaN,3.500000,NaN,2.750000,NaN,3.000000,NaN,NaN,2.875000,NaN,NaN,NaN,NaN,3.250000,NaN,NaN
Belgium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.1250,NaN,NaN,NaN,NaN,2.750000,3.500000,NaN,3.750000,NaN,NaN,2.392857,3.250000,3.611111,NaN,2.333333,NaN,3.500000,3.125000,NaN,NaN,3.750000,NaN,3.000000,NaN,NaN,NaN,NaN,3.250000,NaN,NaN,3.00,NaN,NaN,NaN,NaN
Bolivia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.750000,NaN,NaN,NaN,NaN,NaN,3.750000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Brazil,NaN,NaN,NaN,3.75,NaN,2.750,NaN,NaN,NaN,3.5000,NaN,NaN,NaN,NaN,NaN,3.375000,NaN,NaN,NaN,NaN,3.500000,NaN,3.500000,NaN,NaN,NaN,NaN,3.500000,NaN,NaN,NaN,NaN,3.250000,NaN,NaN,NaN,NaN,3.125000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Canada,2.666667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.50,3.25,3.500000,3.000000,3.250000,3.875000,3.625000,NaN,3.402344,NaN,3.050000,NaN,3.468750,NaN,2.750000,3.375000,2.375000,3.500000,3.750000,NaN,3.041667,NaN,NaN,NaN,NaN,3.083333,NaN,NaN,4.00,NaN,NaN,NaN,NaN
Chile,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.750000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
